In [1]:
import importlib as imp
import sys
import numpy as np
sys.path = list(set(sys.path + ['../../']))

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"
import tensorflow as tf

In [3]:
from sklearn import datasets
dataset_iris = datasets.load_iris(return_X_y=True)

In [4]:
def generate_sample_dataset_and_feature_dict(dataset_iris):
    """
        Make dataset from iris, assuming a user likes class 0 but dislikes class 1 and class 2
    """
    npd = np.zeros(len(dataset_iris[0]), dtype=[
        ('user_id', np.int32), ('item_id', np.int32), ('label', np.int32)])
    fea_dict = dict()
    for idx in range(len(dataset_iris[0])):
        if dataset_iris[1][idx] == 0:
            npd[idx] = (0, idx, 1)
        else:
            npd[idx] = (0, idx, -1)
        fea_dict[idx] = dataset_iris[0][idx]
    return npd, fea_dict

data_all, fea_dict = generate_sample_dataset_and_feature_dict(dataset_iris)

In [5]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data_all)
print(len(data_train), len(data_test))

112 38


In [6]:
# create dataset instance from structured ndarray data
import recsys.dataset as dataset
dataset = imp.reload(dataset)
train_dataset = dataset.Dataset(data_train, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Train')
test_dataset = dataset.Dataset(data_test, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Test')

In [7]:
# create featurizer for mapping: (user, item) -> vector
import recsys.featurizers.plain_featurizer as plain_featurizer
plain_featurizer = imp.reload(plain_featurizer)
class Featurizer(plain_featurizer.PlainFeaturizer):
    """
        Custom featurizer
    """
    def __init__(self, fea_dict):
        self.fea_dict = fea_dict

    def feature_dim(self):
        """
            Dim of X
        """
        return 4
    
    def featurize(self, user_id, item_id):
        if user_id != 0:
            raise "error"
        return self.fea_dict[item_id]

featurizer_iris = Featurizer(fea_dict)

In [8]:
# create data sampler for training and testing
import recsys.samplers.ranknet_sampler as ranknet_sampler
ranknet_sampler = imp.reload(ranknet_sampler)
train_sampler = ranknet_sampler.create_training_sampler(
    dataset=train_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, batch_size=10, num_process=1, seed=100)
test_sampler = ranknet_sampler.create_evaluation_sampler(
    dataset=test_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, seed=10)

In [9]:
# create evaluation metric
import recsys.metrics.auc as auc
auc = imp.reload(auc)
auc_evaluator = auc.AUC()

In [10]:
# create model
import recsys.recommenders.linear_recommender as linear_recommender
linear_recommender = imp.reload(linear_recommender)
model = linear_recommender.LinearRankNetRec(feature_dim=featurizer_iris.feature_dim(),
    init_model_dir=None, save_model_dir='LinearRankNetRec', l2_reg=None, train=True, serve=True)

In [11]:
# create model trainer with model
import recsys.model_trainer as model_trainer
model_trainer = imp.reload(model_trainer)
trainer = model_trainer.ModelTrainer(model=model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
import datetime
print(datetime.datetime.now())

2019-08-02 11:53:56.051252


In [13]:
# train
trainer.train(total_iter=100, 
                    eval_iter=1,
                    save_iter=5,
                    train_sampler=train_sampler,
                    eval_samplers=[test_sampler], 
                    evaluators=[auc_evaluator])

[Training starts, total_iter: 100, eval_iter: 1, save_iter: 5]
[iter 1] loss: 1.901944
..(dataset: Test) evaluation
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 2] loss: 1.403317
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 0.28
[iter 3] loss: 0.511628
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 0.8
[iter 4] loss: 0.270449
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 5] Model saved.
[iter 5] loss: 0.368411
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 6] loss: 0.163296
..(dataset: Test) evaluation
INFO:tensorflow:Restoring par

..(dataset: Test) AUC 1.0
[iter 44] loss: 0.029467
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 45] Model saved.
[iter 45] loss: 0.054966
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 46] loss: 0.013827
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 47] loss: 0.015722
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 48] loss: 0.008149
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 49] loss: 0.009533
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
Instructions for updating:
Use standard file APIs t

..(dataset: Test) AUC 1.0
[iter 87] loss: 0.018192
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 88] loss: 0.006422
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 89] loss: 0.001484
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 90] Model saved.
[iter 90] loss: 0.000845
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 91] loss: 0.007284
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 92] loss: 0.005267
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from LinearRankNetRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 93] loss: 0.009804
..(dataset: Test) evaluati

In [14]:
import datetime
print(datetime.datetime.now())

2019-08-02 11:54:06.782690


In [15]:
# Export Protobuf model for online serving
!rm -rf ./pbModel
model.export(export_model_dir="pbModel", as_text=False)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: pbModel/saved_model.pb


In [16]:
# inspect internal op tensor value of training
b = train_sampler.next_batch()
print("data:", b)
print("dy, dy_tilde:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.get_outputs()))
print("loss:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.get_losses()))

data: [([5.2, 3.5, 1.5, 0.2], [6.8, 2.8, 4.8, 1.4], 1)
 ([6.8, 2.8, 4.8, 1.4], [5.2, 3.5, 1.5, 0.2], 0)
 ([5.2, 3.5, 1.5, 0.2], [5.6, 2.5, 3.9, 1.1], 1)
 ([5.6, 2.5, 3.9, 1.1], [5.2, 3.5, 1.5, 0.2], 0)
 ([5.2, 3.5, 1.5, 0.2], [5.9, 3. , 4.2, 1.5], 1)
 ([5.9, 3. , 4.2, 1.5], [5.2, 3.5, 1.5, 0.2], 0)
 ([5.2, 3.5, 1.5, 0.2], [6.4, 3.2, 5.3, 2.3], 1)
 ([6.4, 3.2, 5.3, 2.3], [5.2, 3.5, 1.5, 0.2], 0)
 ([5.2, 3.5, 1.5, 0.2], [5.1, 2.5, 3. , 1.1], 1)
 ([5.1, 2.5, 3. , 1.1], [5.2, 3.5, 1.5, 0.2], 0)
 ([4.4, 3. , 1.3, 0.2], [6.8, 2.8, 4.8, 1.4], 1)
 ([6.8, 2.8, 4.8, 1.4], [4.4, 3. , 1.3, 0.2], 0)
 ([4.4, 3. , 1.3, 0.2], [5.6, 2.5, 3.9, 1.1], 1)
 ([5.6, 2.5, 3.9, 1.1], [4.4, 3. , 1.3, 0.2], 0)
 ([4.4, 3. , 1.3, 0.2], [5.9, 3. , 4.2, 1.5], 1)
 ([5.9, 3. , 4.2, 1.5], [4.4, 3. , 1.3, 0.2], 0)
 ([4.4, 3. , 1.3, 0.2], [6.4, 3.2, 5.3, 2.3], 1)
 ([6.4, 3.2, 5.3, 2.3], [4.4, 3. , 1.3, 0.2], 0)
 ([4.4, 3. , 1.3, 0.2], [5.1, 2.5, 3. , 1.1], 1)
 ([5.1, 2.5, 3. , 1.1], [4.4, 3. , 1.3, 0.2], 0)]
dy, dy_tilde:

In [17]:
# inspect internal op tensor value of testing
b = test_sampler.next_batch()
print("data:", b)
print("score:", model.serve_inspect_ports(batch_data=b[1], ports=model.servegraph.get_outputs()))

data: ([1, 1, 1, 1, 1, -1, -1, -1, -1, -1], array([([5. , 3.2, 1.2, 0.2],), ([5. , 3.6, 1.4, 0.2],),
       ([5.1, 3.5, 1.4, 0.2],), ([4.6, 3.4, 1.4, 0.3],),
       ([5.4, 3.9, 1.7, 0.4],), ([6.1, 3. , 4.6, 1.4],),
       ([5.5, 2.5, 4. , 1.3],), ([6.8, 3. , 5.5, 2.1],),
       ([6.1, 2.6, 5.6, 1.4],), ([6.1, 3. , 4.9, 1.8],)],
      dtype=[('x', '<f4', (4,))]))
score: [array([[ -6.5172477],
       [ -6.7989345],
       [ -6.900555 ],
       [ -6.3456917],
       [ -7.5059996],
       [-11.834221 ],
       [-10.437129 ],
       [-13.41805  ],
       [-13.295145 ],
       [-12.03035  ]], dtype=float32)]


In [18]:
# inspect internal graph weights
def inspect_weights(checkpoint_model_dir):
    reader = tf.train.NewCheckpointReader(checkpoint_model_dir)
    variables = reader.get_variable_to_shape_map()
    for var, shape in variables.items():
        trained_weight = reader.get_tensor(var)
        print(var)
        print(shape)
        print(trained_weight)
inspect_weights("./LinearRankNetRec/model.ckpt")

interactiongraph/Weights1dTensor/bias__0
[1]
[0.00730727]
interactiongraph/Weights1dTensor/FC__0
[4, 1]
[[-0.99433905]
 [ 0.0218717 ]
 [-1.4521759 ]
 [ 0.59880996]]
